In [ ]:
from Levenshtein import distance as lev
import pandas as pd

# **With the following notebook I achieved the score 3.47360, Hope you will be able to do more good stuffs with this. I am beginner at this field. I know the things I have done are not efficient. If you have any quesitons regarding this, please comment! I will reply them all**

In [ ]:

dictionaryWords = list(pd.read_csv('/kaggle/input/bengali-ged/Bangla Words.csv')['words'].values)
#SOme models
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("sagorsarker/mbert-bengali-ner")
model = AutoModelForTokenClassification.from_pretrained("sagorsarker/mbert-bengali-ner")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)
example = "আমি জাহিদ এবং আমি ঢাকায় বাস করি।"

ner_results = nlp(example)
print(ner_results)

In [ ]:
!pip install banglanltk
import banglanltk as bn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 kB 258.7 kB/s eta 0:00:00


In [ ]:
def Bangla_error_check(gg):
    k = nlp(gg)     # previous pipeline
    #print(k)
    expression_lst = [",","-",":",";"] #things I need to avoid in the sentence

    final = ""
    for elm in k:
        label = elm["entity_group"] #check what group it is in 
        flag = True
        if label not in ["LABEL_1","LABEL_2"]: # Label 1 and label 2 are probable name of a person but it is not always accurate. So if they are name then we will not check them throug
            temp = gg[elm["start"]:elm["end"]] # gatting the word some times the nlp object specify a part of sentence as a word
            label_temp = elm["word"] # Taking this as a label for furthere checking
            #print(label_temp)
            temp = temp.split()  # spliting them into a list, so that if they are part of a sentence then we will get separate word in a list
            #print(temp)

            for word in temp:# looping here words are single word
                #print(word)

                    
                if bn.pos_tag(word)=="CD": # if they are digit we wont check it just will append it
                    final += word + " "
                elif label_temp[0:2] == "##" and flag: #some time nlp can classify suffix in bangla word where it gets split like ##something
                    final = final.rstrip()
                    final += word + " "
                    flag = False

                  

                elif len(word)!= 1: 
                    #word = word.rstrip()
                    if word[-1] in ["।","?","!"]: #checking if the word is the last word in the sentence

                        if word[:-1] in dictionaryWords: # if it in dictionary
                            final += word 
                        elif bn.pos_tag(word)=="CD": # if it is last word also a digit then we are not going to check, we will just append
                            final += word
                        else:
                            final += "$"+ word[:-1] +"$" +word[-1]  # if we can not find it in the dictionary
                    elif word[-1] in expression_lst:#if the word has ",","-",":",";" at the end
                        #print("YO")
                        if word[:-1] in dictionaryWords: # we will check them into the dictionary 
                            final += word +" "
                        elif bn.pos_tag(word[:-1])=="CD": #if they are digit we will just append them 
                            final += word + " "
                        elif word[0] in ["(","{","["] and word[-2] in [")","}","]"]: # if the words has (){}[] on both sides
                                if word[1:-2] in dictionaryWords: # we will check them if they are in the dictionary
                                    final += word +" "
                                elif bn.pos_tag(word[1:-2])=="CD": # if they are digit we will just append them
                                    final += word + " "
                                else:
                                    final +=word[0] +"$"+ word[1:-2] +"$" + word[-2]+word[-1]+" "  #if they are not in dictionary
                        else:
                            final += "$"+ word[:-1] +"$" + word[-1]+" " # if they do not satisfy the conditions
                    elif word[0] in ["(","{","["] and word[-1] in [")","}","]"]: #if the words has (){}[] on both sides
                        if word[1:-1] in dictionaryWords:# we will check them if they are in the dictionary
                            final += word +" "
                        elif bn.pos_tag(word[1:-1])=="CD":# if they are digit we will just append them
                            final += word + " "
                        else:
                            final +=word[0] +"$"+ word[1:-1] +"$" + word[-1]+" "        #else                                 
                    else:

                        if word in dictionaryWords: # here we check the raw word if they do not satisfy the above conditions
                            final += word +" "
                        else:
                            final += "$"+ word +"$" +" "
                else:
                    final = final.rstrip()
                    final += word +' '


        else:

            final += gg[elm["start"]:elm["end"]] +" " # if they are name then we will just append them
    #print(gg)
    final = final.rstrip(" ")
    print(final)
    return final
    

In [ ]:
#(৮০)
#কিন্তু তোমার প্রেমে পড়ি, আমার সিদ্ধান্তের জন্য এখন আমরা একসাথে।
k = "আর, রূপালিও তাই চায়।"
print(Bangla_error_check(k))
k = "পড়ি"
print(k in dictionaryWords)

আর, $রূপালিও$ তাই চায়।
আর, $রূপালিও$ তাই চায়।
True


In [ ]:
abul = pd.read_csv('/kaggle/input/bengali-ged/SampleSubmission.csv')

abul['Expected'] = abul['Expected'].map(Bangla_error_check) 

ব্যক্তি থেকেই শুরু যার সমাপ্তি হবে বিশ্বে মুসলিমদের মহত্ত্বের চর্চায় যেমনটি আগে ছিল $রাসূলের$ আদর্শে আছে।
$ইউনিটগণ$, পিছিয়ে যান।
$ভিক্টোরিয়ার$ $ক্লেটন$ ক্যাম্পাসে ছয়টি আবাসিক হল $রয়েছে$।
তেমনি আবির একটা $বইপোঁকা$ সেটা বললে ভুল $হবেনা$।
আমরা চাই গরিব চাচা যেন $তারনেয্য$ টাকা পিরে পায় এবং সারা $বাংলাদেসে$ যমুনা টিবির সুনাম বয়ে জায়
আমি একজন $আদনান$ ভাইকে $খুজছি$।
এই দুটি শহর শত শত $কিলোমিটারের$ ব্যবধানে অবস্থিত।
খেলার $সময়$ $ক্রোবারকে$ পিছনে রাখা হলেও ছোট $ডেফনিকে$ প্রাধান্য দেন।
এই ভবনটির $স্থাপতি$ ছিলেন আয়ার ল্যান্ডের নাগরিক জেমস হোবান।
এটি হিউ জ্যাকম্যান অভিনীত এবং প্রযোজিত একটি $চলচ্চিত্রে$ অভিযোজিত $হয়েছে$।
আমাকে একেবারে ধ্বংসের $মুখে$ নিয়ে গিয়ে ফেলেছে।
সর্বোপরি তাদেরকে দান করেছেন তিনি তাদের প্রয়োজন অনুসারে সবকিছু।
সেখানে শামসুর রাহমানের কবিতা নিয়ে একটি আলোচনা অনুষ্ঠানের আয়োজন করা হয়।
চা শেষ করে লিলিয়ান তার $বাবার$ কোলে উঠে বসে- তার মাকে দেখতে যাবে।
আমরা দেখতে পাচ্ছি, সুবোধের কারিগর $ভীরু$।
$বিয়ের$ পর তার নাম $হয$ শ্রীমতি লীলাবতী রায যিনি লীলা রায় নামে সমধিক পরিচিত।
মৃত্যুকালে

In [ ]:
#abul.to_csv('SecondAttempt.csv', index = False)

Area of Improvements:
1. Better wordlist
2. Inflection in wordlist
3. Named Entity Recognition
4. Tokenizer
5. Contextual Detection ??